In [ ]:
!mkdir -p ~/.kaggle
#!cp kaggle.json ~/.kaggle
#!chmod 600 ~/.kaggle/kaggle.json
!cp /content/'kaggle.json' ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# API for fetching data to Kaggle :
!kaggle datasets download -d bittlingmayer/amazonreviews

Dataset URL: https://www.kaggle.com/datasets/bittlingmayer/amazonreviews
License(s): unknown
amazonreviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
from zipfile import ZipFile
Amazonedata = '/content/amazonreviews.zip'
with ZipFile(Amazonedata, 'r') as zip:
  zip.extractall()
  print('Amazone data has been extracted')

Amazone data has been extracted


In [ ]:
!pip install datasets


In [ ]:
!pip install evaluate


In [ ]:
# Importing dependency :
from datasets import Dataset
from transformers import AutoModelForSequenceClassification, BertForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, TrainingArguments, Trainer
import evaluate
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
import tensorflow as tf
import re
import string
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers import Layer, Embedding, MultiHeadAttention, Dense, LayerNormalization, Input, Flatten






In [ ]:
import bz2
# Specify the path to the compressed file
compressed_file_path = '/content/train.ft.txt.bz2'

# Decompress the file and read its content
with bz2.open(compressed_file_path, 'rt', encoding='ISO-8859-1') as f:
    lines = f.readlines()

# Parse the lines and create a list of dictionaries
data = []
for line in lines:
    parts = line.strip().split(' ', 1)  # Split into label and text, with a maximum of 1 split
    label = parts[0].replace('__label__', '')  # Remove the prefix '__label__'
    text = parts[1]
    data.append({'label': label, 'text': text})

# Create a DataFrame from the list of dictionaries
Amazone_train = pd.DataFrame(data)

# Display the DataFrame
print(Amazone_train.head())

  label                                               text
0     2  Stuning even for the non-gamer: This sound tra...
1     2  The best soundtrack ever to anything.: I'm rea...
2     2  Amazing!: This soundtrack is my favorite music...
3     2  Excellent Soundtrack: I truly like this soundt...
4     2  Remember, Pull Your Jaw Off The Floor After He...


In [ ]:
df = Amazone_train.head(10000)
print(df.shape)

(10000, 2)


In [ ]:
# Convert label values from strings to integers and subtract 1
df['label'] = df['label'].astype(int) - 1

# Update the 'label' column in the DataFrame with the modified values
df['label'] = df['label'].values


<ipython-input-16-7a0aff43a8a5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['label'].astype(int) - 1
<ipython-input-16-7a0aff43a8a5>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['label'].values


In [ ]:
df.head(3)

,label,text
0,1,Stuning even for the non-gamer: This sound tra...
1,1,The best soundtrack ever to anything.: I'm rea...
2,1,Amazing!: This soundtrack is my favorite music...


#### Starting to connect from here

### Tranformer model using Hugging face

In [ ]:
# Dependencies:
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split

In [ ]:
# Creating custome dataset
class CustomdatasetAmazone(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = torch.tensor(self.labels[idx])

        encoding = self.tokenizer(text, truncation=True, padding="max_length", max_length=self.max_len)
        return {
            'input_ids': encoding['input_ids'],
            'attention_mask': encoding['attention_mask'],
            'label': label
        }

In [ ]:
# prepare tokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
checkpoint = 'distilbert-base-uncased'
device = 'cuda'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# creating model
model = AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=2)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.weight', 'pre_classifier.we

In [ ]:
model # cheking model architecture

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [ ]:
X = df['text'].tolist()
y = df['label'].tolist()


In [ ]:
dataset = CustomdatasetAmazone(X, y, tokenizer)

In [ ]:
dataset[0].keys()

dict_keys(['input_ids', 'attention_mask', 'label'])

In [ ]:
# Spliting training and testing data:
train_amazone, test_amazone = train_test_split(dataset, test_size = 0.3, random_state=42)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
def computing_metrics(example):
  labels = example.label_ids
  preds = example.predictions.argmax(-1)

  f1 = f1_score(labels, preds, average='weighted')
  acc = accuracy_score(labels, preds)

  return {'accuracy': acc, 'f1': f1}

In [ ]:
from transformers import Trainer, TrainingArguments
batch_size = 16
model_name = 'distilbert_finetuned_sentiment'

In [ ]:
args = TrainingArguments(
    output_dir='output',
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=2e-5,
    num_train_epochs=5,
    evaluation_strategy='epoch'
)

In [ ]:
trainer = Trainer(model = model,
                  args = args,
                  train_dataset=train_amazone,
                  eval_dataset=test_amazone,
                  compute_metrics=computing_metrics,
                  tokenizer=tokenizer
                  )

In [ ]:
trainer.train()# training through 30 epochs

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.177882,0.935000,0.934970
2,0.165400,0.249237,0.932667,0.932614
3,0.111200,0.321766,0.930000,0.929962
4,0.049500,0.365720,0.930667,0.930665
5,0.040100,0.443852,0.929333,0.929260
6,0.013000,0.429985,0.938333,0.938323
7,0.010300,0.579970,0.927000,0.926880
8,0.010100,0.502068,0.934333,0.934335
9,0.010100,0.585580,0.930333,0.930233
10,0.011900,0.515840,0.933000,0.932994


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.177882,0.935000,0.934970
2,0.165400,0.249237,0.932667,0.932614
3,0.111200,0.321766,0.930000,0.929962
4,0.049500,0.365720,0.930667,0.930665
5,0.040100,0.443852,0.929333,0.929260
6,0.013000,0.429985,0.938333,0.938323
7,0.010300,0.579970,0.927000,0.926880
8,0.010100,0.502068,0.934333,0.934335
9,0.010100,0.585580,0.930333,0.930233
10,0.011900,0.515840,0.933000,0.932994


TrainOutput(global_step=13140, training_loss=0.018272416448227965, metrics={'train_runtime': 2882.8539, 'train_samples_per_second': 72.844, 'train_steps_per_second': 4.558, 'total_flos': 2.781815371776e+16, 'train_loss': 0.018272416448227965, 'epoch': 30.0})

In [ ]:
# Evaluate the model
eval_metrics = trainer.evaluate()

# Access accuracy
accuracy = eval_metrics["eval_accuracy"]

In [ ]:
print(accuracy) # Accuracy of testing data is highest among models when using fine tunning bert uncased model.

0.9383333333333334
